# Vancouver Neighborhood Accessibility Index

**Author:** Jerry Yang  
**Date:** 2025-07-10

 **Neighborhood Accessibility Index** for the City of Vancouver using public datasets:
- **Schools** (points)
- **Rapid transit stations** (points)
- **Bikeways** (lines)
- **Parks** (polygons)
- **Local area boundaries** (neighbourhood polygons)


In [18]:

!pip install geopandas shapely pyproj scikit-learn h3pandas h3 leafmap folium matplotlib mapclassify -q

#Enviroment setup


In [19]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler
import h3pandas
import leafmap
from pathlib import Path
import h3

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

#city of Vancouver Data API(opendata)
VAN_LOCAL_AREAS_URL="https://opendata.vancouver.ca/api/records/1.0/download?dataset=local-area-boundary&format=geojson"
VAN_PARKS_URL="https://opendata.vancouver.ca/api/records/1.0/download?dataset=parks&format=geojson"
VAN_BIKEWAYS_URL="https://opendata.vancouver.ca/api/records/1.0/download?dataset=bikeways&format=geojson"
VAN_RAPID_STN_URL="https://opendata.vancouver.ca/api/records/1.0/download?dataset=rapid-transit-stations&format=geojson"
VAN_SCHOOLS_URL="https://opendata.vancouver.ca/api/records/1.0/download?dataset=schools&format=geojson"

#CRS_cords

#in meters and 10N
CRS_EQUAL_AREA="EPSG:26910"  
#WGS84 for both H3 polygon and web_based map
CRS_LATLO="EPSG:4326"   

#Load Datasets

In [20]:
def read_geojson(url):
    gdf= gpd.read_file(url)
    return gdf

neighborhoods= read_geojson(VAN_LOCAL_AREAS_URL)     #polygon
parks= read_geojson(VAN_PARKS_URL)                   #polygon
bike_paths= read_geojson(VAN_BIKEWAYS_URL)           #lines
stations= read_geojson(VAN_RAPID_STN_URL)            #point
schools= read_geojson(VAN_SCHOOLS_URL)               #point

display(neighborhoods.head(2))

display(parks.head(2))

display(bike_paths.head(2))

display(stations.head(2))

display(schools.head(2))

Skipping field geo_point_2d: unsupported OGR type: 3
Skipping field googlemapdest: unsupported OGR type: 3
Skipping field geo_point_2d: unsupported OGR type: 3
Skipping field geo_point_2d: unsupported OGR type: 3
Skipping field geo_point_2d: unsupported OGR type: 3


,name,geometry
0,Riley Park,"POLYGON ((-123.10562 49.23312, -123.11617 49.2..."
1,Shaughnessy,"POLYGON ((-123.15527 49.23452, -123.15508 49.2..."


,parkid,official,hectare,neighbourhoodname,facilities,advisories,name,ewstreet,specialfeatures,washrooms,nsstreet,streetname,streetnumber,neighbourhoodurl,geometry
0,8,1,4.85,Arbutus-Ridge,Y,N,Trafalgar Park,W 23rd Avenue,N,Y,Valley Drive,W 23rd Avenue,2610,https://vancouver.ca/news-calendar/arbutus-rid...,POINT (-123.16188 49.25046)
1,20,1,0.85,Downtown,Y,N,Emery Barnes Park,Davie Street,N,N,Richards Street,Richards Street,1170,https://vancouver.ca/news-calendar/downtown.aspx,POINT (-123.124 49.27646)


,segment_length,overall_direction,street_segment_type,bikeway_direction,bike_route_name,subtype,e_s_bound_type,aaa_segment,object_id,bikeway_type,...,snow_removal,surface_type,aaa_network,street_name,w_n_bound_type,upgrade_year,construction_note,vehicle_direction,speed_limit,geometry
0,149.282645,EW,Arterial,2W,SW Marine,PBT,Painted Lanes,NO,1015,Painted Lanes,...,Yes,Paved,NO,SW Marine Drive,Painted Lanes,2016,upgrade to wide paint,2W,50.0,"LINESTRING (-123.1464 49.20862, -123.14702 49...."
1,54.981381,NS,Residential,2W,Chilco,None,Local Street,YES,1020,Local Street,...,No,Paved,YES,Chilco,Local Street,2021,Upgrade to AAA in 2021,2W,30.0,"LINESTRING (-123.13732 49.29409, -123.13785 49..."


,station,geo_local_area,geometry
0,King Edward,Riley Park,POINT (-123.11534 49.24912)
1,Burrard,Downtown,POINT (-123.11997 49.28586)


,geo_local_area,school_category,address,school_name,geometry
0,Downtown,Independent School,688 W Hastings St,Alexander Academy,POINT (-123.11401 49.285)
1,Killarney,StrongStart BC,3340 E 54th Av,Captain James Cook StrongStart Centre,POINT (-123.03583 49.21938)


#CRS targetting

In [21]:
targets=[("neighborhoods", neighborhoods),
           ("parks", parks), ("bike_paths", bike_paths),
           ("stations", stations), ("schools", schools)]
proj={}
for name, gdf in targets:
    proj[name]=gdf.to_crs(CRS_EQUAL_AREA)

neighborhoods= proj["neighborhoods"].reset_index(drop=True)

parks= proj["parks"]

bike_paths= proj["bike_paths"]

stations= proj["stations"]

schools= proj["schools"]

print("Working CRS:", neighborhoods.crs)

Working CRS: EPSG:26910


#Computing vectorized indicators

In [22]:
#stable Ids
neighborhoods['neighbor_id']= neighborhoods.index

#Points within polygons
schools_in= gpd.sjoin(schools, neighborhoods[['neighbor_id','geometry']], how='inner', predicate='within')

stns_in= gpd.sjoin(stations, neighborhoods[['neighbor_id','geometry']], how='inner', predicate='within')

schools_cnt= schools_in.groupby('neighbor_id').size().rename('schools')

stns_cnt= stns_in.groupby('neighbor_id').size().rename('rapid_stations')

#Line clipping
bike_clip= gpd.overlay(bike_paths, neighborhoods[['neighbor_id','geometry']], how='intersection')

bike_clip['seg_len_m']= bike_clip.length

bike_len = bike_clip.groupby('neighbor_id')['seg_len_m'].sum().rename('bike_length_in_m')

#Poly to Poly
parks_clip= gpd.overlay(parks, neighborhoods[['neighbor_id','geometry']], how='intersection')

parks_clip['section_area_m2'] = parks_clip.area

park_area= parks_clip.groupby('neighbor_id')['section_area_m2'].sum().rename('park_area_m2')

#Join metrics

metrics = pd.concat([schools_cnt, stns_cnt, bike_len, park_area], axis=1).fillna(0.0)

neighborhoods= neighborhoods.merge(metrics, on='neighbor_id', how='left')

neighborhoods[['neighbor_id','schools','rapid_stations','bike_length_in_m','park_area_m2']].head()

,neighbor_id,schools,rapid_stations,bike_length_in_m,park_area_m2
0,0,4,1.0,11408.330515,0.0
1,1,10,0.0,8453.371318,0.0
2,2,10,0.0,14637.288359,0.0
3,3,7,0.0,12564.387148,0.0
4,4,7,8.0,29005.501316,0.0


#Normalization,Smoothing,Composite Index

In [31]:


if "neigh_id" not in neighborhoods.columns and "neighbor_id" in neighborhoods.columns:
    neighborhoods= neighborhoods.rename(columns={"neighbor_id": "neigh_id"})

if "bike_length_m" not in neighborhoods.columns and "bike_length_in_m" in neighborhoods.columns:
    neighborhoods= neighborhoods.rename(columns={"bike_length_in_m": "bike_length_m"})
#column checking
need_cols = ["schools", "rapid_stations", "bike_length_m", "park_area_m2"]
for c in need_cols:
    if c not in neighborhoods.columns:
        raise KeyError(f"Missing column: {c}. Go back to Section 5 and make sure it's created.")

#Unit checking(met to num, 0 filling empty)
neighborhoods["schools"]= pd.to_numeric(neighborhoods["schools"], errors="coerce").fillna(0)

neighborhoods["rapid_stations"]= pd.to_numeric(neighborhoods["rapid_stations"], errors="coerce").fillna(0)

neighborhoods["bike_length_m"]= pd.to_numeric(neighborhoods["bike_length_m"], errors="coerce").fillna(0)

neighborhoods["park_area_m2"]= pd.to_numeric(neighborhoods["park_area_m2"], errors="coerce").fillna(0)

# normalization from 0 to 1, min-,max
for col in need_cols:
    col_min = neighborhoods[col].min()
    
    col_max = neighborhoods[col].max()
    
    col_range = col_max - col_min
    if col_range == 0:
        neighborhoods[col + "_norm"]= 0.0
    else:
        neighborhoods[col + "_norm"]= (neighborhoods[col] - col_min) / col_range

#smoothing neighbors, if no neighbors then keep it as its original normalized val
smooth_cols = ["schools_smooth", "rapid_stations_smooth", "bike_length_m_smooth", "park_area_m2_smooth"]

#filling the empty array
schools_s= []

stns_s= []

bike_s= []

parks_s= []

for i in neighborhoods.index:
    this_geom= neighborhoods.loc[i, "geometry"]

    #masking for whether neighbor is touching currnet one
    touches_mask= neighborhoods.geometry.touches(this_geom)

    #exclusion of the current one
    touches_mask.loc[i]= False

    #pick rows of touching neighbors
    touching_rows = neighborhoods.loc[touches_mask, :]

    #no neighbors:keep own normalized value
    if touching_rows.shape[0] == 0:
        
        schools_s.append(neighborhoods.loc[i, "schools_norm"])
        
        stns_s.append(neighborhoods.loc[i, "rapid_stations_norm"])
        
        bike_s.append(neighborhoods.loc[i, "bike_length_m_norm"])
        
        parks_s.append(neighborhoods.loc[i, "park_area_m2_norm"])
        
    # average of neighbors' normalized values
    else:
        
        schools_s.append(touching_rows["schools_norm"].mean())
        
        stns_s.append(touching_rows["rapid_stations_norm"].mean())
        
        bike_s.append(touching_rows["bike_length_m_norm"].mean())
        
        parks_s.append(touching_rows["park_area_m2_norm"].mean())

#save smoothed columns
neighborhoods["schools_smooth"]= schools_s

neighborhoods["rapid_stations_smooth"]= stns_s

neighborhoods["bike_length_m_smooth"]= bike_s

neighborhoods["park_area_m2_smooth"]= parks_s




#composite indexing by choosing weight
#comments: based off personal preference transit weights over other factor
w_schools = 0.25
w_transit = 0.35
w_bike    = 0.20
w_parks   = 0.20

neighborhoods["access_index"] = (
    w_schools * neighborhoods["schools_smooth"]
    
  + w_transit * neighborhoods["rapid_stations_smooth"]
  
  + w_bike    * neighborhoods["bike_length_m_smooth"]
  
  + w_parks   * neighborhoods["park_area_m2_smooth"]
)

#finalized spatial data with smoothed idnex val
finalized_column= [
    ("neigh_id" if "neigh_id" in neighborhoods.columns else "neighbor_id"),
    "access_index",
    "schools","rapid_stations","bike_length_m","park_area_m2",
    "schools_norm","rapid_stations_norm","bike_length_m_norm","park_area_m2_norm",
    "schools_smooth","rapid_stations_smooth","bike_length_m_smooth","park_area_m2_smooth"
]
finalized_column= [c for c in finalized_column if c in neighborhoods.columns]

display(neighborhoods[finalized_column].head(100))


,neigh_id,access_index,schools,rapid_stations,bike_length_m,park_area_m2,schools_norm,rapid_stations_norm,bike_length_m_norm,park_area_m2_norm,schools_smooth,rapid_stations_smooth,bike_length_m_smooth,park_area_m2_smooth
0,0,0.205036,4,1.0,11408.330515,0.0,0.055556,0.125,0.208343,0.0,0.388889,0.125000,0.320320,0.0
1,1,0.145537,10,0.0,8453.371318,0.0,0.388889,0.000,0.075406,0.0,0.263889,0.062500,0.288447,0.0
2,2,0.285487,10,0.0,14637.288359,0.0,0.388889,0.000,0.353606,0.0,0.597222,0.187500,0.352780,0.0
3,3,0.160759,7,0.0,12564.387148,0.0,0.222222,0.000,0.260351,0.0,0.277778,0.000000,0.456572,0.0
4,4,0.149073,7,8.0,29005.501316,0.0,0.222222,1.000,1.000000,0.0,0.200000,0.075000,0.364113,0.0
5,5,0.131273,9,1.0,16810.300028,0.0,0.333333,0.125,0.451365,0.0,0.222222,0.083333,0.232756,0.0
6,6,0.120557,7,2.0,9648.318664,0.0,0.222222,0.250,0.129164,0.0,0.250000,0.041667,0.217370,0.0
7,7,0.150480,8,0.0,16926.048070,0.0,0.277778,0.000,0.456572,0.0,0.259259,0.000000,0.428324,0.0
8,8,0.201869,7,1.0,16485.592410,0.0,0.222222,0.125,0.436757,0.0,0.166667,0.250000,0.363511,0.0
9,9,0.232824,17,1.0,16904.999754,0.0,0.777778,0.125,0.455626,0.0,0.460317,0.142857,0.338724,0.0


#Mapping

In [24]:
#using WGS84 for web tiles
CRS_LATLON= "EPSG:4326"
neighborhoods_ll= neighborhoods.to_crs(CRS_LATLON).copy()


index_col = "access_index"

#build popup field list
desired_fields= [
    "neigh_id",
    index_col,
    "schools",
    "rapid_stations",
    "bike_length_m",
    "park_area_m2",
]
popup_fields = [c for c in desired_fields if c in neighborhoods_ll.columns]

print("Styling column:", index_col)
print("Popup fields:", popup_fields)

#map
m= leafmap.Map(center=[49.25, -123.12], zoom=11, draw_control=False, measure_control=False)
m.add_basemap("CartoDB.Positron")

m.add_data(
    neighborhoods_ll,
    column=index_col,
    scheme="Quantiles",
    k=5,
    cmap="GnBu",
    legend_title="Accessibility Index (Neighborhoods)",
    fields=popup_fields,
)
m


Styling column: access_index
Popup fields: ['neigh_id', 'access_index', 'schools', 'rapid_stations', 'bike_length_m', 'park_area_m2']


Map(center=[49.25, -123.12], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

#H3 GRID

In [44]:
#convert right index for H3 (WGS84)
neighborhoods_wgs84=neighborhoods.to_crs(CRS_LATLON)

boundings=neighborhoods_wgs84.total_bounds
print(f"Vancouver bounds:{boundings}")

H3_RES=8

#generating H3
def h3_for_boundings(boundings,resolution):
    minx,miny,maxx,maxy=boundings
    
    hexa=set()
    
    lat_gap=int((maxy-miny)*100)
    lon_gap=int((maxx-minx)*100)
    
    #dense sampling for each h3 cell
    for i in range(lat_gap + 1):
        for j in range(lon_gap + 1):
            lat = miny + (maxy - miny) * i / lat_gap
            lon = minx + (maxx - minx) * j / lon_gap
            hex_id = h3.geo_to_h3(lat, lon, resolution)
            hexa.add(hex_id)
            
    return list(hexa)

print(f"Generating H3 cells at resolution {H3_RES}...")

van_h3= h3_for_boundings(boundings, H3_RES)
print(f"Generated {len(van_h3)} H3 cells.")

#H3 to GeoDataFrame
def h3_to_gdf(hex_list):
    geometries=[]
    hex_ids=[]
    for hex_id in hex_list:
        boundary= h3.h3_to_geo_boundary(hex_id, geo_json=True)
        polygon= Polygon(boundary)
        geometries.append(polygon)
        hex_ids.append(hex_id)
    gdf= gpd.GeoDataFrame({'h3_id': hex_ids, 'geometry': geometries}, crs=CRS_LATLON)
    return gdf

van_h3_grid= h3_to_gdf(van_h3)

#Bounding box filtering
print("Filtering H3 cells to Vancouver boundary...")
vancouver_boundary=neighborhoods_wgs84.unary_union
van_h3_grid_filtered= van_h3_grid[van_h3_grid.intersects(vancouver_boundary)]
print(f"Filtered to {len(van_h3_grid_filtered)} H3 cells within Vancouver boundary.")

#H3 grid analysis
print("Analyzing H3 cells for accessibility metrics...")
van_h3_grid_mapped= van_h3_grid_filtered.to_crs(CRS_EQUAL_AREA)
van_h3_grid_mapped['h3_index']=van_h3_grid_mapped.index

    #calculating Accessibility index for each H3 cell:

        ##schools count
print("Calculating accessibility index for each H3 cell...")
schools_in_h3=gpd.sjoin(schools, van_h3_grid_mapped[['h3_index','geometry']], how='inner', predicate='within')
schools_in_h3_cnt= schools_in_h3.groupby('h3_index').size().rename('schools')

        ##rapid transit stations count
print("Calculating rapid transit stations count...")
stations_in_h3=gpd.sjoin(stations, van_h3_grid_mapped[['h3_index','geometry']], how='inner', predicate='within')
stations_in_h3_cnt= stations_in_h3.groupby('h3_index').size().rename('rapid_stations')
        
        ##bike path length
print("Calculating bike path length...")
bike_clip_h3= gpd.overlay(bike_paths, van_h3_grid_mapped[['h3_index','geometry']], how='intersection')
bike_clip_h3['seg_len_m']= bike_clip_h3.length
bike_len_h3= bike_clip_h3.groupby('h3_index')['seg_len_m'].sum().rename('bike_length_m')

        ##park area
print("Calculating park area...")
parks_in_h3 = gpd.sjoin(parks, van_h3_grid_mapped[['h3_index','geometry']], 
                        how='inner', predicate='within')
parks_count_h3 = parks_in_h3.groupby('h3_index').size().rename('parks_count_h3')

#joining metrics
h3_metrics= pd.concat([schools_in_h3_cnt, stations_in_h3_cnt, bike_len_h3, parks_count_h3], axis=1).fillna(0.0)

van_h3_grid_mapped= van_h3_grid_mapped.merge(h3_metrics, on='h3_index', how='left')
van_h3_grid_mapped=van_h3_grid_mapped.fillna(0)
van_h3_grid_mapped[['h3_index','schools','rapid_stations','bike_length_m','parks_count_h3']].head()

print("H3 calculation completed.")

#Normalization
print("Normalizing H3 metrics...")
columns_to_normalize= ['schools', 'rapid_stations', 'bike_length_m', 'parks_count_h3']

for col in columns_to_normalize:
    col_min= van_h3_grid_mapped[col].min()
    col_max= van_h3_grid_mapped[col].max()
    col_range= col_max - col_min
    if col_range == 0:
        van_h3_grid_mapped[col + '_norm']= 0.0
    else:
        van_h3_grid_mapped[col + '_norm']= (van_h3_grid_mapped[col] - col_min) / col_range
        
print("Normalization completed.")

#Same neighbourhood weights
w_schools = 0.25
w_transit = 0.35
w_bike    = 0.20
w_parks   = 0.20

van_h3_grid_mapped['access_index']= (
    w_schools * van_h3_grid_mapped['schools_norm'] +
    w_transit * van_h3_grid_mapped['rapid_stations_norm'] +
    w_bike    * van_h3_grid_mapped['bike_length_m_norm'] +
    w_parks   * van_h3_grid_mapped['parks_count_h3_norm']
)

print("Accessibility index calculation completed.")

#H3 map visualization
print("Visualizing H3 accessibility index on map...")
van_h3_grid_wgs84= van_h3_grid_mapped.to_crs(CRS_LATLON)

m_h3= leafmap.Map(center=[49.25, -123.12], zoom=12, draw_control=False, measure_control=False)
m_h3.add_basemap("CartoDB.Positron")

h3_popup_fields= ['h3_id', 'access_index', 'schools', 'rapid_stations', 'bike_length_m', 'parks_count_h3']
m_h3.add_data(
    van_h3_grid_wgs84,
    column='access_index',
    scheme='Quantiles',
    k=5,
    cmap="YlOrRd",
    legend_title=f'H3 Accessibility (Res {H3_RES})',
    fields=h3_popup_fields,
    layer_name='H3 Accessibility Index'
)

print("Map visualization completed.")
m_h3

Vancouver bounds:[-123.22484589   49.1989368  -123.02320099   49.2958107 ]
Generating H3 cells at resolution 8...
Generated 190 H3 cells.
Filtering H3 cells to Vancouver boundary...
Filtered to 149 H3 cells within Vancouver boundary.
Analyzing H3 cells for accessibility metrics...
Calculating accessibility index for each H3 cell...
Calculating rapid transit stations count...
Calculating bike path length...
Calculating park area...
H3 calculation completed.
Normalizing H3 metrics...
Normalization completed.
Accessibility index calculation completed.
Visualizing H3 accessibility index on map...


Map visualization completed.


Map(center=[49.25, -123.12], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

#Exporting

In [ ]:
out_fp = OUT_DIR / "vancouver_neighborhood_access_index.geojson"

neighborhoods.to_crs(CRS_LATLON).to_file(out_fp, driver="GeoJSON")

print("Exported:", out_fp.resolve())

Exported: C:\Users\yfz10\OneDrive\桌面\spatial project\spatial-analysis-project\outputs\vancouver_neighborhood_access_index.geojson
